In [ ]:
from copy import deepcopy
from os import environ
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer
from requests import get, Response
from hashlib import sha256
from tqdm.notebook import tqdm
from zipfile import ZipFile
from IPython.display import display, Markdown
import pandas as pd
import missingno as msno
import seaborn as sns
import numpy as np
from IPython.display import clear_output
from lightgbm import LGBMClassifier
from lightgbm import LGBMClassifier

import re
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
# Added imports for the new transformer
from scipy import stats
from random import sample

from lightgbm import LGBMClassifier
from deliverables.utils.image_inverter import save
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate, GridSearchCV, \
    ParameterGrid
from mlflow import set_tracking_uri, log_metrics, log_figure
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Partie 0 - Constantes, imports et outils

In [ ]:
_cache_folder = Path('~/.cache/gn_p7').expanduser()
_cache_folder.mkdir(parents=True, exist_ok=True)

_ds_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip'

graph_folder: Path = Path("./graphs")
random_state: int = 42


def save_figure(figure: plt.Figure, folder: str, figure_name: str) -> None:
    folder = graph_folder / folder
    folder.mkdir(parents=True, exist_ok=True)
    save(figure, folder / f'{figure_name}.png', close=True)


def download(url: str) -> Path:
    url_id: str = sha256(url.encode('utf-8')).hexdigest()
    local_path: Path = _cache_folder / url_id
    local_path.parent.mkdir(parents=True, exist_ok=True)
    if not local_path.exists():
        tmp_path: Path = _cache_folder / (url_id + '.tmp')
        res: Response = get(url, stream=True)
        with tmp_path.open('wb') as f, tqdm(
                total=int(res.headers.get('content-length')),
                desc=f'Downloading {url}',
                unit_scale=True) as q:
            for chunk in res.iter_content(chunk_size=8192):
                q.update(len(chunk))
                f.write(chunk)
        tmp_path.replace(local_path)
    return local_path


def download_zip_archive(url: str) -> Path:
    """Download a zip archive, extract it then return the folder containing its content"""
    archive_path: Path = download(url)
    archive_folder: Path = Path(archive_path.as_posix() + '.dir')

    if not archive_folder.exists():
        print(f'Extracting archive {url}...', flush=True)
        archive_temp: Path = Path(archive_path.as_posix() + '.tmp')
        archive_temp.mkdir(parents=True, exist_ok=True)
        archive: ZipFile = ZipFile(archive_path)
        archive.extractall(path=archive_temp)
        archive_temp.replace(archive_folder)
        print(f'Extracting archive {url}...done', flush=True)

    return archive_folder


datasets: dict[str, pd.DataFrame] = {}


def get_dataset(name: str) -> pd.DataFrame:
    folder = download_zip_archive(_ds_url)
    if not name.endswith('.csv'):
        name = f'{name}.csv'
    try:
        return datasets[name]
    except KeyError:
        try:
            _df = pd.read_csv(folder / name)
        except FileNotFoundError:
            display(Markdown(f'# ERROR: Dataset {name!r} not found, available datasets are:\n' + '\n'.join(
                f'- {p.name}' for p in sorted(folder.iterdir(), key=(lambda x: x.name.lower())))))
            raise KeyError(name) from None
        else:
            datasets[name] = _df
            return _df.copy()


# Partie 1 - EDA

## Partie 1.0 - Chargement des données

In [ ]:
from sklearn.model_selection import train_test_split

train, test = map(get_dataset, ('application_train', 'application_test'))
train_ratio = len(train) / (len(train) + len(test))
test_ratio = len(test) / (len(train) + len(test))
train_df, test_df = train, test = train_test_split(
    train,
    test_size=test_ratio,
    random_state=random_state,
    stratify=train['TARGET']
)

In [ ]:
# Séparation des features et de la cible
X = train_df.drop('TARGET', axis=1)
y = train_df['TARGET']

print("Données d'entraînement chargées :", X.shape)
print("Cible chargée :", y.shape)

In [ ]:
# Séparation des features et de la cible
X = test_df.drop('TARGET', axis=1)
y = test_df['TARGET']

print("Données de test chargées :", X.shape)
print("Cible chargée :", y.shape)

In [ ]:
train

In [ ]:
test

## Partie 1.1 - Analyse de la cible

La cible est présente dans le dataset d'entrainement mais pas dans le dataset de test, pour éviter les fuites de données.

In [ ]:
save_figure(train.TARGET.value_counts().plot.pie(
    title='Répartition des cibles (0=paiement complet, 1=retards de paiement)'
).figure, '1_model', '0_target')

## Partie 1.3 - Analyse des features (hors cible)

In [ ]:
# Function to calculate missing values by column# Funct
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
                                                              "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns


def missing_stats():
    for fold in ('train', 'test'):
        msno.matrix(get_dataset('application_' + fold + '.csv'), fontsize=12)
        plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
        save_figure(plt.gcf(), '1_model', '1_missing_' + fold.title())

    for fold in ('train', 'test'):
        df = get_dataset('application_' + fold + '.csv')
        msno.matrix(df[list(sorted(df.columns, key=(lambda col: int(df[col].notna().sum()))))], fontsize=12)
        plt.title(f'Missing Values Count ({fold.title()}ing fold)', fontsize=16)
        save_figure(plt.gcf(), '1_model', '2_sorted_missing_' + fold.title())

    for fold in ('train', 'test'):
        missing_test_values = missing_values_table(get_dataset('application_' + fold + '.csv'))

        # TODO: Set the plot style for dark mode when exporting to png
        plt.figure(figsize=(16, 12))  # There are a lot of columns
        sns.barplot(x=missing_test_values['% of Total Values'], y=missing_test_values.index)
        plt.title(f'Percentage of Missing Values by Feature ({fold.title()}ing fold)', fontsize=16)
        plt.xlabel('% of Total Values', fontsize=12)
        plt.ylabel('Features', fontsize=12)

        # Add percentage text on the bars
        for index, value in enumerate(missing_test_values['% of Total Values']):
            plt.text(value, index, f' {value}%', va='center')

        plt.xlim(0, 110)  # Set x-limit to give space for text
        plt.tight_layout()
        save_figure(plt.gcf(), '1_model', '3_graph_missing' + fold.title())


missing_stats()

Nous pouvons voir qu'à peu près la moitié des colonnes manquent au moins une valeur, et que le reste est défini à environ 45-75%
Si nous nous intéressons

In [ ]:
assert not len(train.columns[
                   (train.dtypes != 'int64') &
                   (train.dtypes != 'float64') &
                   (train.dtypes != 'object')]), 'Plus de types de colonnes sont présentes'
display(Markdown('Il existe trois types de données en entrée, int64 et float64, numériques, et object, catégorielles'))
display(
    Markdown('Il arrive parfois que des données numériques soient accidentellement catégorisées en "object" si elles'
             ' contiennent des valeurs non numérique, ce n\'est pas le cas ici'))

## Partie 2 - Définition du Score Métier

Nous somme face à deux déséquilibres:
- Un déséquilibre des coûts (pb métier) : un faux négatif coûtant dix fois plus cher qu'un faux positif, il faut en tenir compte pour minimiser plus fortement les faux négatifs.
    - Nous devons créer un `scorer` pour Scikit-Learn qui minimise le coût `coût = 10 * FN + 1 * FP`
    - Il doit aussi trouver le seuil de décision optimal, car le seuil par défaut de 0.5 n'est probablement pas le meilleur d'un point de vue métier.
- Un déséquilibre des classes (pb données) : 92% des cibles étant négatives, un risque serai accru d'avoir un modèle prédisant trops 0 (prédire 0 tout le temps nous donnera 92% de précision.
    - Cela sera traité en utilisant comme paramètre `class_weight='balanced'`
    - TODO: Don't forget to include it every time I introduce a new model

In [ ]:
def find_optimal_threshold(y_true: np.ndarray, y_pred_proba: np.ndarray):
    """Trouve le seuil qui minimise le coût métier."""
    # Si y_pred_proba est 2D, on prend la probabilité de la classe 1
    if y_pred_proba.ndim == 2:
        y_scores = y_pred_proba[:, 1]
    # Sinon, on suppose que c'est déjà le score pour la classe positive
    else:
        y_scores = y_pred_proba

    thresholds = np.linspace(0.01, 0.99, 100)
    costs = []
    for t in thresholds:
        y_pred = (y_scores >= t).astype(int)  # On utilise y_scores ici
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
        costs.append(10 * fn + 1 * fp)

    optimal_t_index = np.argmin(costs)
    return thresholds[optimal_t_index], costs[optimal_t_index]


def business_cost_scorer_func(y_true: np.ndarray[..., ...], y_pred_proba: np.ndarray[..., ...]):
    """Calcule le coût métier minimal en trouvant le meilleur seuil."""
    _, min_cost = find_optimal_threshold(y_true, y_pred_proba)
    return min_cost


def optimal_threshold_scorer_func(y_true: np.ndarray[..., ...], y_pred_proba: np.ndarray[..., ...]):
    """Retourne le seuil optimal qui minimise le coût métier."""
    optimal_t, _ = find_optimal_threshold(y_true, y_pred_proba)
    return optimal_t


# Création des scorers pour Scikit-Learn
# On veut MINIMISER le coût, donc greater_is_better=False
business_scorer = make_scorer(business_cost_scorer_func, greater_is_better=False, response_method='predict_proba')

# Pour le seuil, c'est juste une information, donc pas de notion de 'meilleur'
threshold_scorer = make_scorer(optimal_threshold_scorer_func, response_method='predict_proba')

# On définit le dictionnaire de scoring qu'on utilisera dans la cross-validation
scoring = {
    'business_cost': business_scorer,
    'business_threshold': threshold_scorer,
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall'
}
print("Scorers métier créés avec succès.")

# Partie 3 - Définition d'un pipeline de prétraitement

L'avantage que le pipeline de prétraitement a est la robustesse contrer le data leakage, en effet les modèles d'apprentissages et de traitement seront entraîné sur les mêmes données, ce qui sera obligatoire pour pouvoir utiliser des techniques de K Fold en s'assurant que les folds soient indépendants les uns des autres.

Note: Nous allons ignorer la transformation BoxCox en premier lieu

Note relatives au déséquilibre de la cible:<br>
Il existe un déséquilibre flagrant entre 1 et 0, avec 1 n'étant présent que pour 8% des données d'entraînement.<br>
Pour y remédier, on a a notre disposition:
- oversampling, par exemple SMOT: Synthetic Minority OverSampling Technique
- undersampling
- cost sensitive learning: Ajuster le scoring pour pénaliser davantage les faux négatifs



TODO: Revoir s'il est nécessaire de tester les trois, ou si cost sensitive learning est suffisant


In [ ]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from functools import wraps
from typing import Callable, Any


class OccupationPCA(BaseEstimator, TransformerMixin):
    """
    ORGANIZATION_TYPE et OCCUPATION_TYPE donnent 78 colonnes après OneHot, ce qui complexifie le dataset.
    En théorie cela nous donne 37 colonnes post pca
    Pour minimiser le nombre de features, nous allons procéder de la manière suivante:
    1. Application d'un One-Hot Encoding sur les colonnes 'ORGANIZATION_TYPE' et 'OCCUPATION_TYPE'.
    2. Application d'une ACP pour conserver 95% de la variance, en nommant composantes 'OT_0', 'OT_1', etc.
    5. Suppression des colonnes d'origine et ajout des nouvelles composantes au DataFrame.
    """

    def __init__(self, n_components=0.95):
        self.n_components = n_components  # Will either be a variance or number of components
        self.pca = PCA(n_components=self.n_components)  # This will be "fitted" with the estimator

        # To make sure the dataset holds the same shape, even if some values are missing from the testing dataset
        self.dummy_columns = []

    # TODO: Dummy NA?
    def fit(self, X, y=None):
        x = pd.get_dummies(X[['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']], dummy_na=True)
        self.dummy_columns = x.columns
        self.pca.fit(x)
        return self

    def transform(self, X):
        x = X.copy()
        x_org = pd.get_dummies(x[['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']], dummy_na=True)

        # Add, remove and move around columns to get the same shape as the training set
        x_org = x_org.reindex(columns=self.dummy_columns, fill_value=0)

        pca_result = self.pca.transform(x_org)  # Apply pre-trained PCA

        # Turning the results into a dataframe to include to x
        pca_df = pd.DataFrame(pca_result, columns=[f"OT_{i}" for i in range(pca_result.shape[1])], index=x.index)

        # Replacing old values with new pca values
        return pd.concat([x.drop(columns=['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']), pca_df], axis=1)


class CategoricalEncoder(BaseEstimator, TransformerMixin):

    def __init__(self, handle_unknown='ignore', sparse_output=False):
        self.handle_unknown = handle_unknown
        self.sparse_output = sparse_output
        self.ohe = OneHotEncoder(handle_unknown=self.handle_unknown,
                                 sparse_output=self.sparse_output)

    def fit(self, X, y=None):
        self.categorical_columns_ = X.select_dtypes(include=['object']).columns.tolist()
        if self.categorical_columns_:
            self.ohe.fit(X[self.categorical_columns_])
        return self

    def transform(self, X):
        X_transformed = X.copy()
        if self.categorical_columns_:
            encoded_data = self.ohe.transform(X[self.categorical_columns_])
            new_cols = self.ohe.get_feature_names_out(self.categorical_columns_)
            encoded_df = pd.DataFrame(encoded_data, columns=new_cols, index=X.index)
            X_transformed = X_transformed.drop(columns=self.categorical_columns_)
            X_transformed = pd.concat([X_transformed, encoded_df], axis=1)
        return X_transformed


def copy_and_return_x[T, U](callback: Callable[[T, U], None]) -> Callable[[T, U], T]:
    @wraps(callback)
    def wrapped(x: T, y: U = None) -> T:
        callback((x := x.copy()), y)
        return x

    return wrapped


@FunctionTransformer
@copy_and_return_x
def custom_preprocessor(X, y=None) -> None:
    # On commence par réparer l'erreur que nous avons vu à l'étape d'Analyse Exploratoire
    X['DAYS_EMPLOYED'] = X['DAYS_EMPLOYED'].replace({365243: np.nan})

    # On va ensuite supprimer la variable identifiant
    X.pop('SK_ID_CURR')  # TODO: Check for this

    # On va enfin traiter les valeurs catégorielles

    # TODO: Vaut-il mieux utiliser trois colonnes?
    X['CODE_GENDER'] = X['CODE_GENDER'].map({'F': 1, 'M': -1}).fillna(0)

    for flag_name in X.columns[X.columns.str.startswith('FLAG_')]:
        values = set(X[flag_name])
        if {0, 1} - values:
            continue
        if {"Y", "N"} - values:
            X[flag_name] = X[flag_name].map({'Y': 1, 'N': 0})
        else:
            raise RuntimeError(flag_name, values)


@FunctionTransformer
@copy_and_return_x
def sanitize_feature_names(x, y=None):
    x.columns = [re.sub(r'[^A-Za-z0-9_]+', '', str(col)) for col in x.columns.tolist()]


# 2. Build the pipeline
preprocessing_pipeline = Pipeline([
    ('custom_preprocessor', custom_preprocessor),
    ('occupation_pca', OccupationPCA()),
    ('first_name_sanitization', deepcopy(sanitize_feature_names)),
    ('dummy_preprocessor', CategoricalEncoder(handle_unknown='ignore', sparse_output=False)),
    ('second_name_sanitization', deepcopy(sanitize_feature_names)),
], verbose=True)

# We try to see if the pipeline works as expected
X_train = train.copy()
y_train = X_train.pop('TARGET')
deepcopy(preprocessing_pipeline).fit_transform(X_train, y_train)

# Partie 4: Outils

In [ ]:
# TODO: Danger of this, if values are negatives and test values are smaller than train values, offset won't be enough
# I don't want the BoxCox transformer to mess up data that it cannot fix.
# TODO: Review how I estimate if it's worth the transformation
epsilon = 1e-6


class ConditionalBoxCox(BaseEstimator, TransformerMixin):

    def __init__(self, p_value_threshold: float = 0.05):
        self.p_value_threshold = p_value_threshold
        self.columns_to_transform_ = []
        self.learned = {}

    def _as_numpy(self, X: pd.DataFrame | np.ndarray[tuple[int, ...], np.dtype[np.float64]]):
        x: np.ndarray
        if isinstance(X, pd.DataFrame):
            for col in X.columns:
                if X[col].isna().any():
                    raise NotImplementedError(
                        'I do not know yet how this would be handled,\n\t'
                        'for now, you need this to follow an inputer to have no null values')
            columns_to_drop = [i for i in X.columns if X[i].dtype == 'object']
            columns_to_keep = [i for i in X.columns if X[i].dtype in ('float64', 'int64')]
            if len(other_columns := set(X.columns) - set(columns_to_keep) - set(columns_to_drop)):
                raise NotImplementedError(other_columns)
            x = X.to_numpy()
        elif isinstance(X, np.ndarray):
            x = X
        else:
            raise NotImplementedError(type(X))
        return x

    # noinspection PyTypeHints
    def fit(self, X: pd.DataFrame | np.ndarray[tuple[int, ...], np.dtype[np.float64]], y=None):
        x = self._as_numpy(X.copy())

        for col, data in enumerate(x.T):
            if len(set(data)) < 10:  # Not worth the change
                continue
            data = data + (shift := epsilon - (0 if (data > 0).all() else data.min()))
            _, initial_p = stats.shapiro(sample(list(data), 100))
            transformed_data, ld = stats.boxcox(data)
            _, transformed_p = stats.shapiro(sample(list(transformed_data), 100))
            if transformed_p > initial_p and transformed_p > self.p_value_threshold:
                self.columns_to_transform_.append(col)
                self.learned[col] = ld, shift
        return self

    def transform(self, X):
        x = self._as_numpy(X.copy())
        for index in self.columns_to_transform_:
            ld, shift = self.learned[index]
            x[:, index] = stats.boxcox(
                x[:, index] + shift, lmbda=ld)
        return x


def new_pipeline(*steps):
    return Pipeline(deepcopy(preprocessing_pipeline.steps) + list(steps))

## Partie 5: Pipelines de modélisation

Nous allons maintenant définir nos modèles et lancer les expérimentations. Chaque modèle sera testé dans un `run` MLflow distinct.

Pour chaque `run`, nous allons logger :
1.  **Les paramètres** : Tous les hyperparamètres du pipeline.
2.  **Les métriques** : Les scores (AUC, coût métier, seuil) calculés par validation croisée (moyenne et écart-type).
3.  **Le modèle final** : Le pipeline complet, entraîné sur l'ensemble des données, prêt à être déployé.
4.  **(Optionnel) Des artefacts** : Comme des graphiques (ex: feature importance).

In [ ]:
from mlflow import set_experiment, start_run, log_params, log_metric, log_param, models, sklearn

# TODO: `random_state`

# Définition des modèles de prétraitement
pipelines = [
    ('DummyClassifier', new_pipeline(('model', DummyClassifier(strategy='stratified'))), {}),
    ('LogisticRegression', new_pipeline(
        ('imputer', SimpleImputer()),
        ('coxbox', ConditionalBoxCox()),
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(class_weight='balanced'))), {
         'model__C': [0.1, 1.0, 10.0], 'model__max_iter': [100, 1000]}),
    ('LightGBMClassifier', new_pipeline(('model', LGBMClassifier(class_weight='balanced'))), {
        'model__n_estimators': [100, 200], 'model__learning_rate': [0.05, 0.1], 'model__num_leaves': [20, 31]})
]
pipelines = [pipelines[-1]]

# Définition de la stratégie de validation croisée
cv_strategy = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=random_state)

X_train = train.copy()
y_train = X_train.pop('TARGET')
deepcopy(preprocessing_pipeline).fit_transform(X_train, y_train)
X_train = train.copy()
y_train = X_train.pop('TARGET')

display(Markdown(f"***{len(pipelines)}*** modèles sont prêts à être testés."))

# MLFlow
set_tracking_uri('http://127.65.12.247:50222')
set_experiment("P7_Credit_Scoring_Models")

best_global_score = float('inf')  # On veut minimiser le coût, donc on part de l'infini
best_model = None  # Cette variable stockera le meilleur pipeline trouvé
best_model_name = ""
best_params = {}

# Boucle d'expérimentation
for model_name, pipeline_base, grid_params in pipelines:
    print(f"--- Lancement du run pour le modèle : {model_name} ---")

    grid = list(ParameterGrid(grid_params)) or [{}]  # In case there are no hyper parameters

    # Cette boucle manuelle me permet de logger toutes les métrique, et non pas uniquement les bonnes
    with start_run(run_name=model_name) as run:

        local_best_global_score = float('inf')  # On veut minimiser le coût, donc on part de l'infini
        local_best_model = None  # Cette variable stockera le meilleur pipeline trouvé
        local_best_model_name = ""
        local_best_params = {}

        for parameters_index, parameters_data in enumerate(grid):
            run_name: str = f'{model_name}_run_{parameters_index + 1}'

            print(f"  > Run: {run_name} avec les paramètres : {parameters_data}")

            pipeline = deepcopy(pipeline_base)

            log_params(parameters_data)

            pipeline.set_params(**parameters_data)

            cv_results = cross_validate(
                estimator=pipeline,
                X=X_train,
                y=y_train,
                cv=cv_strategy,
                scoring=scoring,
                n_jobs=-1,
                return_train_score=False  # TODO: Should I return those as well?
            )

            # 4. Log metrics for EACH FOLD to see stability
            for scorer_name in scoring.keys():
                for fold_index, fold_score in enumerate(cv_results[f'test_{scorer_name}']):
                    # Log each fold's score as a step in the metric's history
                    log_metric(key=f"{scorer_name}_per_fold", value=fold_score, step=fold_index)

            # 5. Log SUMMARY metrics (mean, std) for easy comparison
            summary_metrics = {}
            for scorer_name in scoring.keys():
                mean_score = np.mean(cv_results[f'test_{scorer_name}'])
                std_score = np.std(cv_results[f'test_{scorer_name}'])

                summary_metrics[f'mean_{scorer_name}'] = mean_score
                summary_metrics[f'std_{scorer_name}'] = std_score

            summary_metrics['mean_fit_time'] = np.mean(cv_results['fit_time'])
            log_metrics(summary_metrics)
            current_cost = summary_metrics['mean_business_cost']
            if current_cost < best_global_score:
                print(f"  > Nouveau meilleur modèle trouvé ! Coût: {current_cost:.2f}")
                best_global_score = current_cost
                best_model_name = run_name
                best_model = deepcopy(pipeline_base)
                best_params = parameters_data

            if current_cost < local_best_global_score:
                print(f"  > Nouveau meilleur modèle trouvé ! Coût: {current_cost:.2f}")
                local_best_global_score = current_cost
                local_best_model_name = run_name
                local_best_model = deepcopy(pipeline_base)
                local_best_params = parameters_data

            print(f"  > Run {run_name} terminé. Coût métier moyen: {-summary_metrics['mean_business_cost']:.2f}")
        model = deepcopy(local_best_model)
        model.set_params(**local_best_params)
        X_test = test.copy()
        y_test = X_test.pop('TARGET')
        model.fit(X_train, y_train)
        sklearn.log_model(pipeline, name=local_best_model_name, signature=models.infer_signature(
            X_test.head(), model.predict(X_test.head()), local_best_params))
    print(f"--- Run pour {model_name} terminé et loggé. ---\n")

with start_run(run_name=best_model_name) as run:
    model = deepcopy(best_model)
    model.set_params(**best_params)
    X_test = test.copy()
    y_test = X_test.pop('TARGET')
    model.fit(X_train, y_train)
    sklearn.log_model(pipeline, name=best_model_name, signature=models.infer_signature(
        X_test.head(), model.predict(X_test.head()), best_params))

print("\n--- FIN DE LA RECHERCHE ---")
print(f"Le meilleur modèle global est : {best_model_name} avec un coût métier de {best_global_score:.2f}")

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
X_train = train.copy()
y_train = X_train.pop('TARGET')
best_model.fit(X_train, y_train)
X_test = test.copy()
y_test = X_test.pop('TARGET')
RocCurveDisplay.from_estimator(best_model, X_test, y_test, ax=ax)
plt.title("Courbe ROC du meilleur modèle")

# Sauvegarde la figure en tant qu'artefact MLflow
log_figure(fig, "roc_curve.png")

plt.show()

In [ ]:
import mlflow
import mlflow.sklearn
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, RocCurveDisplay, roc_auc_score, accuracy_score, f1_score
from copy import deepcopy

# --- 1. Préparation des données ---
# Assurons-nous d'avoir des jeux de données propres pour l'entraînement final et le test
X_train = train.drop('TARGET', axis=1)
y_train = train['TARGET']
X_test = test.drop('TARGET', axis=1)
y_test = test['TARGET']

# --- 2. Création et entraînement du modèle final ---
print("--- Entraînement du modèle final sur l'ensemble des données d'entraînement ---")
final_model = deepcopy(best_model)
final_model.set_params(**best_params)
final_model.fit(X_train, y_train)
print("Entraînement terminé.")

# --- 3. Démarrage d'un run MLflow dédié au modèle champion ---
with mlflow.start_run(run_name=f"{best_model_name}_CHAMPION") as run:
    run_id = run.info.run_id
    print(f"Run MLflow démarré avec succès. Run ID: {run_id}")
    print("Logging des informations du modèle champion...")

    # --- 4. Log des hyperparamètres et des tags ---
    mlflow.log_params(best_params)
    mlflow.set_tag("model_status", "champion")
    mlflow.set_tag("training_data_shape", X_train.shape)

    # --- 5. Évaluation sur le jeu de test et log des métriques ---
    y_pred_proba_test = final_model.predict_proba(X_test)

    # Récupération du seuil métier optimal calculé sur le jeu d'entraînement (pratique standard)
    # ou recalculé sur le test pour voir la performance pure (ici on le recalcule pour l'info)
    optimal_threshold_test = scoring['business_threshold']._score_func(y_test, y_pred_proba_test)
    business_cost_test = scoring['business_cost']._score_func(y_test, y_pred_proba_test)

    # Prédictions de classe avec le seuil optimisé
    y_pred_class_test = (y_pred_proba_test[:, 1] >= optimal_threshold_test).astype(int)

    # Calcul des métriques
    test_metrics = {
        "test_business_cost": business_cost_test,
        "test_optimal_threshold": optimal_threshold_test,
        "test_roc_auc": roc_auc_score(y_test, y_pred_proba_test[:, 1]),
        "test_accuracy_at_threshold": accuracy_score(y_test, y_pred_class_test),
        "test_f1_at_threshold": f1_score(y_test, y_pred_class_test)
    }
    mlflow.log_metrics(test_metrics)
    print("Métriques sur le jeu de test logguées :", test_metrics)

    # --- 6. Création et log des artefacts visuels ---

    # Matrice de confusion
    fig_cm, ax_cm = plt.subplots()
    cm = confusion_matrix(y_test, y_pred_class_test)
    sns.heatmap(cm, annot=True, fmt='d', ax=ax_cm, cmap='Blues')
    ax_cm.set_title(f'Matrice de Confusion (Seuil = {optimal_threshold_test:.2f})')
    ax_cm.set_xlabel('Prédiction')
    ax_cm.set_ylabel('Vraie valeur')
    mlflow.log_figure(fig_cm, "test_confusion_matrix.png")
    plt.close(fig_cm)

    # Courbe ROC
    fig_roc, ax_roc = plt.subplots()
    RocCurveDisplay.from_estimator(final_model, X_test, y_test, ax=ax_roc)
    ax_roc.set_title("Courbe ROC sur le jeu de test")
    mlflow.log_figure(fig_roc, "test_roc_curve.png")
    plt.close(fig_roc)

    print("Artefacts visuels (Matrice de Confusion, Courbe ROC) loggués.")

    # Analyse SHAP (peut prendre un peu de temps)
    print("Calcul des valeurs SHAP pour l'interprétabilité...")
    # Il faut appliquer le preprocessing avant de le passer à l'explainer
    # On récupère le pipeline de preprocessing et le modèle
    preprocessor = final_model[:-1] # Toutes les étapes sauf la dernière ('model')
    model_step = final_model.named_steps['model']

    X_test_transformed = preprocessor.transform(X_test)

    # SHAP a besoin de noms de features valides
    X_test_transformed.columns = [f'feature_{i}' for i in range(X_test_transformed.shape[1])]

    explainer = shap.TreeExplainer(model_step)
    shap_values = explainer(X_test_transformed)

    # Graphique d'importance globale
    fig_shap, ax_shap = plt.subplots()
    shap.summary_plot(shap_values, X_test_transformed, show=False, max_display=20)
    plt.title("Importance globale des features (SHAP)")
    # Ajuster la mise en page pour éviter que les labels ne soient coupés
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "test_shap_summary.png")
    plt.close(fig_shap)
    print("Graphique SHAP loggué.")

    # --- 7. Log du modèle final ---
    # La signature permet à MLflow de connaître les types d'entrée et de sortie
    signature = mlflow.models.infer_signature(X_train.head(), final_model.predict_proba(X_train.head()))

    mlflow.sklearn.log_model(
        sk_model=final_model,
        artifact_path="credit_scoring_model",
        signature=signature,
        registered_model_name=f"{best_model_name}_champion_model" # Enregistre directement le modèle dans le Model Registry
    )
    print("Modèle final loggué et enregistré dans le Model Registry.")

print("\n--- OPÉRATION TERMINÉE ---")
print(f"Le modèle champion a été entraîné, évalué et toutes ses informations sont disponibles dans le run ID: {run_id}")

## Partie 4 - Analyse des Résultats et Prochaines Étapes

Toutes nos expérimentations sont maintenant enregistrées. Pour les visualiser, lance l'interface utilisateur de MLflow via "http://127.65.12.247:50222"


Dans l'interface, tu pourras :
- **Comparer les modèles** en les sélectionnant et en cliquant sur "Compare".
- **Trier les runs** par `business_cost_mean` (le score le plus bas est le meilleur) ou `roc_auc_mean`.
- **Inspecter chaque run** pour voir les paramètres exacts qui ont produit un certain résultat.

### Prochaines étapes basées sur ces résultats :

1.  **Choisir le meilleur modèle** : Sur la base du score métier, sélectionne le modèle le plus prometteur (probablement LightGBM).
2.  **Optimisation des hyperparamètres** : Pour le modèle choisi, tu pourras lancer une nouvelle série d'expérimentations (par exemple avec `GridSearchCV` ou `Hyperopt`) pour trouver les meilleurs hyperparamètres, toujours en optimisant le score métier. Chaque essai de la recherche d'hyperparamètres peut être un `run` MLflow imbriqué !
3.  **Analyse de l'importance des features** : Une fois le modèle finalisé, utilise des librairies comme **SHAP** pour analyser l'importance des features globale et locale, comme demandé dans le brief.
4.  **Enregistrement du modèle final** : Le meilleur run pourra être promu dans le "Model Registry" de MLflow, lui donnant un statut (Staging, Production) pour le déploiement.